<a href="https://colab.research.google.com/github/thileshs/thileshLLM1/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
from langchain import PromptTemplate, LLMChain
from PyPDF2 import PdfReader
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.llms import VertexAI

In [ ]:
import os
import pandas as pd
from PyPDF2 import PdfReader
from langchain import PromptTemplate, LLMChain
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter

# Set Google Application Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/lumen-b-ctl-047-e2aeb24b0ea0 5 2.json"

# CSV and PDF file paths
csv_path = "/content/new_filtered 2.csv"
pdf_paths = [
    '/content/Attempt to fetch past end of record - CST.CST_ACCOUNT 1.pdf',
    '/content/Attempt to fetch past end of record - CST.CST_DAILY_CASHDRAWER 1.pdf',
    '/content/Attempt to fetch past end of record 2.pdf',
    '/content/Number area code not found 3.pdf',
    '/content/The result of a singleton select returned more than 1 value (1) 2.pdf'
]


# Function to extract text from PDFs
def extract_text_from_pdfs(pdf_paths):
    raw_text = ''
    for pdf_path in pdf_paths:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            content = page.extract_text()
            if content:
                raw_text += content
    return raw_text

# Load the CSV file into a DataFrame with the specified encoding
error_logs_df = pd.read_csv(csv_path, encoding='latin1')
def should_drop(value):
    return str(value).startswith('GT2') or str(value).startswith('GTW')

# Drop rows where any of the specified columns start with 'GT2' or 'GTW'
error_logs_df = error_logs_df[~(error_logs_df['Prog'].apply(should_drop) |
                                error_logs_df['Program'].apply(should_drop) |
                                error_logs_df['API'].apply(should_drop))]
# Text processing from PDFs
raw_text = extract_text_from_pdfs(pdf_paths)
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=200, length_function=len)
texts = text_splitter.split_text(raw_text)

# Template for generating prompts
prompt_template = """
You are an AI assistant that summarizes error logs and provides resolutions. Here are some example resolutions extracted from a PDF:
{pdf_text}

---
Read the PDF and analyze it. Now we have the dataframe error_logs_df with columns -
["Log No", "Time", "Prog", "Program", "Section", "Prog Mess", "System messages", "Table"].
In that dataframe, the "Prog Mess" column has some values. Take those values and compare them with
the issues available in the PDF.

For each log, you must:
    1. Check if the table data in the "Table" column matches any table mentioned in the PDF text.
    2. If a match is found, provide a detailed resolution based on the table and the issue.
    3. If no match is found, simply state "No data found".

Now, given the following error log, provide a detailed resolution:

Strictly print Only the log no, Error, issues,Table and resolution.
Error: {error_id}
Resolution: {error_description}
"""

# Function to generate prompt string
def generate_prompt(pdf_text, error_id, error_description):
    return prompt_template.format(pdf_text=pdf_text, error_id=error_id, error_description=error_description)

# Initialize LLMChain with VertexAI language model
llm_chain = LLMChain(
    prompt=PromptTemplate(input_variables=["prompt"], template="{prompt}"),
    llm=VertexAI(model_name="gemini-1.5-pro-preview-0409", temperature=0.5, max_output_tokens=8192)
)

# Function to handle error logs and generate resolution
def handle_error_log(error_id, error_description):
    prompt_string = generate_prompt(raw_text, error_id, error_description)
    generated_resolution = llm_chain.run({"prompt": prompt_string})
    return generated_resolution

# Iterate through each row in the DataFrame and print log details with resolution
for index, row in error_logs_df.iterrows():
    log_id = row['Log No']
    error = row['Prog Mess']

    # Check if log_id is null or blank
    if pd.isna(log_id) or log_id == '':
        print(f"End of CSV reached at row {index}. Stopping further processing.")
        break

    # Generate resolution for the current log entry
    generated_resolution = handle_error_log(log_id, error)

    # Print the formatted output
    print(f"Resolution for Log ID: {log_id}\n{generated_resolution}")
    print("\n---\n")
    print("The Mail been sent to SUPPORT TEAM")

<ipython-input-16-ac91003f4085>:34: DtypeWarning: Columns (1,2,4,5,8,9,10,11,12,14,15,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  error_logs_df = pd.read_csv(csv_path, encoding='latin1')


Resolution for Log ID: 73007643.0
Log no: 73007643.0
Error: %RDB-E-STREAM_EOF, attempt to fetch past end of record stream
Issues: Unable to sign off service  order
Table: CST.CST_ACCOUNT 
Resolution: This resolution is for CST.CST_ACCOUNT Table associated with Attempt to fetch past end of record. There is some data mismatch or unavailable in the database. 


---

The Mail been sent to SUPPORT TEAM
Resolution for Log ID: 73007644.0
Log No: 73007644.0
Error: %RDB-E-STREAM_EOF, attempt to fetch past end of record stream
Issues: Unable to sign off service order
Table: CST.CST_ACCOUNT 
Resolution: This resolution is for CST.CST_ACCOUNT Table associated with Attempt to fetch past end of record. There is some data mismatch or unavailable in the database. 


---

The Mail been sent to SUPPORT TEAM
Resolution for Log ID: 73007645.0
Log No: 73007645.0
Error: %RDB-E-STREAM_EOF, attempt to fetch past end of record stream
Issues: Unable to sign off service order
Table: CST.CST_ACCOUNT 
Resolution: 

In [ ]:
import os
import pandas as pd
from langchain import PromptTemplate, LLMChain
from PyPDF2 import PdfReader
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.llms import VertexAI



os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/lumen-b-ctl-047-e2aeb24b0ea0 5 2.json'



# Load CSV and process dataframe
csv_path = "/content/new_filtered 2.csv"
error_logs_df = pd.read_csv(csv_path)


def should_drop(value):
    return str(value).startswith('GT2') or str(value).startswith('GTW')



error_logs_df = error_logs_df[~(error_logs_df['Prog'].apply(should_drop) |
                                error_logs_df['Program'].apply(should_drop) |
                                error_logs_df['API'].apply(should_drop))]



# Load and process PDF files
pdf_paths = [
    '/content/Attempt to fetch past end of record - CST.CST_ACCOUNT 1.pdf',
    '/content/Attempt to fetch past end of record - CST.CST_DAILY_CASHDRAWER 1.pdf',
    '/content/Attempt to fetch past end of record 2.pdf',
    '/content/Number area code not found 3.pdf',
    '/content/The result of a singleton select returned more than 1 value (1) 2.pdf'
]



raw_text = ''
for pdf_path in pdf_paths:
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        content = page.extract_text()
        if content:
            raw_text += content



# Split text into chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2000,
    chunk_overlap=2,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)



# Create embeddings and FAISS index
embeddings = VertexAIEmbeddings()
document_search = FAISS.from_texts(texts, embeddings)



# Define the prompt template
prompt_template = """
You are an AI assistant that summarizes error logs and provides resolutions. Here are some example resolutions extracted from a PDF:



{pdf_text}



---
Read the PDF and analyze it. Now we have the dataframe `error_logs_df` with columns -
["Log No", "Time", "Prog", "Program", "Section", "Prog Mess", "System messages", "Table"].
In that dataframe, the "Prog Mess" column has some values. Take those values and compare them with
the issues available in the PDF.





Now, given the following error log, provide a detailed resolution:



Strictly print Only the table,Error, issues, and resolution.
Error: {error_id}
Resolution: {error_description}
"""



# Generate the prompt
def generate_prompt(pdf_text, error_id, error_description):
    return prompt_template.format(pdf_text=pdf_text, error_id=error_id, error_description=error_description)



template = PromptTemplate(input_variables=["prompt"], template="{prompt}")
llm_chain = LLMChain(prompt=template, llm=VertexAI(temperature=0.7))



# Handle each error log and find resolution using document search
def handle_error_log(error_id, error_description):
    # Use document search to find the most relevant text chunks from the PDFs
    results = document_search.similarity_search(error_description, k=3)
    relevant_text = "\n".join([result.page_content for result in results])



    # Generate the prompt with the relevant text
    prompt_string = generate_prompt(relevant_text, error_id, error_description)
    generated_resolution = llm_chain.run({"prompt": prompt_string})
    return generated_resolution



for index, row in error_logs_df.iterrows():
    error_id = row['Log No']
    error_description = row['Prog Mess']

    # Check if log number is null or NaN
    if pd.isnull(error_id) or error_id == '':
        print("Log No is null or NaN. Stopping the iteration.")
        break

    resolution = handle_error_log(error_id, error_description)
    print(f"Resolution for Log ID {error_id}:\n", resolution)

<ipython-input-17-805f7cc8887a>:18: DtypeWarning: Columns (1,2,4,5,8,9,10,11,12,14,15,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  error_logs_df = pd.read_csv(csv_path)


Resolution for Log ID 73007643.0:
  | Table | Error | Issues | Resolution |
|---|---|---|---|
| CST.CST_ACCOUNT | 73007643.0 | Unable to sign off service order | This resolution is for CST_Account Table associated with Attempt to fetch past end of record. There is some data mismatch or unavailable in database. |
| CST.CST_DAILY_CASHDRAWER | 73007643.0 | Unable to sign off service order | This resolution is for CST. CST_DAILY_CASHDRAWER Table associated with Attempt to fetch past end of record. There is some data
Resolution for Log ID 73007644.0:
  | Table | Error | Issue | Resolution |
|---|---|---|---|
| CST.CST_ACCOUNT | Attempt to fetch past end of record | Unable to sign off service order | This resolution is for CST_Account Table associated with Attempt to fetch past end of record. There is some data mismatch or unavailable in database. |
| CST. CST_DAILY_CASHDRAWER | Attempt to fetch past end of record | Unable to sign off service order | This resolution is for CST. CST_DAILY_CAS

In [ ]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.2 MB/s eta 0:00:00
